In [2]:
import numpy as np
import cv2
import face_recognition

In [ ]:
# pip install face recognition

img_bgr = face_recognition.load_image_file('image/image5.jpg')
img_rgb = cv2.cvtColor(img_bgr,cv2.COLOR_BGR2RGB)

cv2.imshow('bgr',img_bgr)
cv2.imshow('rgb',img_rgb)

cv2.waitKey(0)
cv2.destroyAllWindows()


In [3]:
## finding face locations and drawing Bounding Boxes

img = face_recognition.load_image_file('image/image5.jpg')
img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
img = cv2.resize(img,(360,360))

# finding face locations for BB
face = face_recognition.face_locations(img)[0]

copy = img.copy()

# drawing rectangles
cv2.rectangle(copy,(face[3],face[0]),(face[1],face[2]),(255,0,255),2)

cv2.imshow('copy',copy)
cv2.imshow('burry',img)

cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
## training an image for Face Recognition

train_encodings = face_recognition.face_encodings(img)[0]

In [ ]:
## testing on Image

test = face_recognition.load_image_file('image/image4.jpg')
test = cv2.cvtColor(test,cv2.COLOR_BGR2RGB)
test = cv2.resize(test,(360,360))

test_encode = face_recognition.face_encodings(test)[0]
print(face_recognition.compare_faces([train_encodings],test_encode))

In [1]:
## Building face recognition system

import cv2
import numpy as np
import face_recognition
import os
import pickle
from datetime import datetime

# function to loop in all the images in the folder and append .jpg from their names

path = 'images'
images = []
classNames = []

myList = os.listdir(path)
# print(myList)
for cl in myList:
    curImg = cv2.imread(f'{path}/{cl}')
    images.append(curImg)
    classNames.append(os.path.splitext(cl)[0])
# print(classNames)

In [2]:
## function to encode all images and store them in a variable \

from codecs import EncodedFile


def findEncoding(images):
    encodingList = []
    for img in images:
        img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        img = cv2.resize(img,(360,360))
        encoded_face = face_recognition.face_encodings(img)[0]
        encodingList.append(encoded_face)
    return encodingList

encode_train = findEncoding(images)

In [3]:
## Creating attendance.csv file to store the attendance with time

def attendance(name):
    with open('attendance.csv','r+') as f:
        myDataList = f.readlines()
        nameList = []
        for line in myDataList:
            entry = line.split(',')
            nameList.append(entry[0])
        if name not in nameList:
            now = datetime.now()
            time = now.strftime('%I:%M:%S:%p')
            date = now.strftime('%d-%B-%Y')
            f.writelines(f'n{name}, {time}, {date}')


In [4]:
# Real webcam for Real-time recognition

cap = cv2.VideoCapture(0)

while True:

    success,img = cap.read()
    imgs = cv2.resize(img,(0,0),None,0.25,0.25)
    imgs = cv2.cvtColor(imgs,cv2.COLOR_BGR2RGB)
    loc_face = face_recognition.face_locations(imgs)
    encode_face = face_recognition.face_encodings(imgs,loc_face)

    for face_encode,face_loc in zip(encode_face,loc_face):
        match = face_recognition.compare_faces(encode_train,face_encode)
        faceDist = face_recognition.face_distance(encode_train,face_encode)
        matchIndex = np.argmin(faceDist)
        # print(matchIndex)

        if match[matchIndex]:
            name = classNames[matchIndex].upper().lower()
            y1,x2,y2,x1 = face_loc


            # since we have scaled down 3 times
            y1,x2,y2,x1 = y1*4,x2*4,y2*4,x1*4
            cv2.rectangle(img,(x1,y1),(x2,y2),(0,255,0),2)
            cv2.rectangle(img,(x1,y2-35),(x2,y2),(0,255,0),cv2.FILLED)
            cv2.putText(img,name,(x1+6,y2-5),cv2.FONT_HERSHEY_COMPLEX,1,(255,255,255),2)

            attendance(name)
    cv2.imshow('output',img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break



: 